In [1]:
from dask.distributed import Client, progress

client = Client(n_workers=4)

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 20,Total memory: 31.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54335,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 31.73 GiB
Comm: tcp://127.0.0.1:54363,Total threads: 5
Dashboard: http://127.0.0.1:54364/status,Memory: 7.93 GiB
Nanny: tcp://127.0.0.1:54338,


**Zadanie 1**  
Poniżej zaprezentowany kod jest podejściem sekwencyjnym do wykonania zadania przetworzenia logów (parsowanie, konwersja daty) i w takiej formie nie można zrównoleglić go tak jak zostało to zaprezentowane na przykładzie powyżej. Przekształcenie łańcucha daty na obiekt datetime wymaga najpierw wykonania parsowania pliku. **Zastanów się i spróbuj przerobić to rozwiązanie tak, aby możliwe było użycie wywołań dask delayed w celu zrównoleglenia części funkcji, np. parsowanie danych w celu pobrania wartości kolumn niezależnie (tylko jednej na raz). Dane końcowe możesz zapisać do dask DataFrame, a następnie do plików parquet.**

In [3]:
import dask
import dask.dataframe as dd
from datetime import datetime
import pandas as pd


def load_logs(path):
    with open(path, 'r') as file:
        return file.readlines()


def parse_date(inp: str):
    date_start = inp.find('[') + 1
    date_end = inp.find(']')
    date_str = inp[date_start:date_end]
    return date_str


def parse_level(inp: str):
    level_start = inp.find('[', inp.find(']')) + 1
    level_end = inp.find(']', level_start)
    return inp[level_start:level_end]


def parse_client(inp: str):
    client_start = inp.find('[', inp.find(']', inp.find(']') + 1))
    if client_start == -1:
        return ""
    client_end = inp.find(']', client_start)
    return inp[client_start + 8: client_end]


def parse_message(inp: str):
    client_start = inp.find('[', inp.find(']', inp.find(']') + 1))
    client_end = inp.find(']', client_start)
    
    if client_start == -1:
        return inp[inp.find(']', inp.find(']')) + 2:]
    else:
        return inp[client_end + 2:]


def convert_date(date_str):
    return datetime.strptime(date_str, "%a %b %d %H:%M:%S %Y")


lines = load_logs('../data/Apache_2k.log')

delayed_dates = [dask.delayed(parse_date)(line) for line in lines]
delayed_levels = [dask.delayed(parse_level)(line) for line in lines]
delayed_clients = [dask.delayed(parse_client)(line) for line in lines]
delayed_messages = [dask.delayed(parse_message)(line) for line in lines]
delayed_converted_dates = [dask.delayed(convert_date)(date) for date in delayed_dates]

output = [dask.delayed(list)(record) for record in zip(delayed_converted_dates, delayed_levels, delayed_clients, delayed_messages)]
results = dask.compute(*output)

In [4]:
df = pd.DataFrame(results, columns=["date", "level", "client", "message"])
df

,date,level,client,message
0,2005-12-04 04:47:44,notice,,[notice] workerEnv.init() ok /etc/httpd/conf/w...
1,2005-12-04 04:47:44,error,,[error] mod_jk child workerEnv in error state 6\n
2,2005-12-04 04:51:08,notice,,[notice] jk2_init() Found child 6725 in scoreb...
3,2005-12-04 04:51:09,notice,,[notice] jk2_init() Found child 6726 in scoreb...
4,2005-12-04 04:51:09,notice,,[notice] jk2_init() Found child 6728 in scoreb...
...,...,...,...,...
1995,2005-12-05 19:14:11,error,,[error] mod_jk child workerEnv in error state 6\n
1996,2005-12-05 19:15:55,notice,,[notice] jk2_init() Found child 6791 in scoreb...
1997,2005-12-05 19:15:55,notice,,[notice] jk2_init() Found child 6790 in scoreb...
1998,2005-12-05 19:15:57,notice,,[notice] workerEnv.init() ok /etc/httpd/conf/w...


In [5]:
df_dd = dd.from_pandas(df, npartitions=4)
df_dd.head()

,date,level,client,message
0,2005-12-04 04:47:44,notice,,[notice] workerEnv.init() ok /etc/httpd/conf/w...
1,2005-12-04 04:47:44,error,,[error] mod_jk child workerEnv in error state 6
2,2005-12-04 04:51:08,notice,,[notice] jk2_init() Found child 6725 in scoreb...
3,2005-12-04 04:51:09,notice,,[notice] jk2_init() Found child 6726 in scoreb...
4,2005-12-04 04:51:09,notice,,[notice] jk2_init() Found child 6728 in scoreb...


In [6]:
df_dd.to_parquet('data')

**Zadanie 2**  
Wykorzystując przykłady zaprezentowane w labie wykonaj na danych `people` (możesz zmniejszyć lub zwiększyć ich wolumen w zależności od potrzeb) operację z użyciem Dask bag, która polegać będzie na przetworzeniu wszystkich plików i zapisaniu do plików o nazwie `expired_{partition}.json` rekordów, których ważność karty kredytowej wygasła (jest to wartość w formacie miesiąc/rok). Zapisując ustaw finalną liczbę plików na 10 jeżeli była inna. Możesz to zrobić poprzez zmianę ilość partycji dask bag (patrz link do API na początku laba).

In [7]:
def is_card_expired(record):
    date = record.get('credit-card', {}).get('expiration-date')

    if date:
        try:
            month, year = map(int, date.split('/'))
            year += 2000
            expiry_date = datetime(year, month, 1)
            return expiry_date < datetime.now()
        except ValueError:
            return False
    return False

In [8]:
import json
import os

b = dask.datasets.make_people(npartitions=100, records_per_partition=20000)

expired_cards = b.filter(is_card_expired)
expired_cards_repartitioned = expired_cards.repartition(10)
expired_cards_repartitioned.map(json.dumps).to_textfiles(os.path.join('data', 'expired_*.json'))

['C:/Users/Arek/PycharmProjects/BDA/Lab03 - dask bags, delayed, futures/data/expired_0.json',
 'C:/Users/Arek/PycharmProjects/BDA/Lab03 - dask bags, delayed, futures/data/expired_1.json',
 'C:/Users/Arek/PycharmProjects/BDA/Lab03 - dask bags, delayed, futures/data/expired_2.json',
 'C:/Users/Arek/PycharmProjects/BDA/Lab03 - dask bags, delayed, futures/data/expired_3.json',
 'C:/Users/Arek/PycharmProjects/BDA/Lab03 - dask bags, delayed, futures/data/expired_4.json',
 'C:/Users/Arek/PycharmProjects/BDA/Lab03 - dask bags, delayed, futures/data/expired_5.json',
 'C:/Users/Arek/PycharmProjects/BDA/Lab03 - dask bags, delayed, futures/data/expired_6.json',
 'C:/Users/Arek/PycharmProjects/BDA/Lab03 - dask bags, delayed, futures/data/expired_7.json',
 'C:/Users/Arek/PycharmProjects/BDA/Lab03 - dask bags, delayed, futures/data/expired_8.json',
 'C:/Users/Arek/PycharmProjects/BDA/Lab03 - dask bags, delayed, futures/data/expired_9.json']

**Zadanie 3**  
Wybierając z danych `people` dane tylko osób dorosłych (zaprezentowane w przykładach w tym labie) przechowaj je w obiekcie typu `bag`, a następnie zapisz je do dask dataframe za pomocą metody `to_dataframe` (pamiętaj o tym jaka jest aktualna struktura pojedynczego rekordu), a następnie zapisz do jednego pliku w formacie parquet.

In [9]:
adults_bag = b.filter(lambda record: record['age'] >= 18)

In [10]:
def to_dataframe_record(record):
    df_record = {
        'first_name': record['name'][0],
        'last_name': record['name'][1],
        'age': record['age'],
        'occupation': record['occupation'],
        'telephone': record['telephone'],
        'address': record['address']['address'],
        'city': record['address']['city'],
        'credit_card_number': record['credit-card']['number'],
        'credit_card_expiration': record['credit-card']['expiration-date']
    }
    return df_record

In [11]:
df = adults_bag.map(to_dataframe_record).to_dataframe()
df.to_parquet('data/adults')
df.head()

,first_name,last_name,age,occupation,telephone,address,city,credit_card_number,credit_card_expiration
0,Florinda,Landry,62,Restaurateur,+18163563228,1000 Emil Terrace,Redding,2605 8637 6783 1544,05/22
1,Joane,Golden,96,Landlord,+13184830272,828 Hamerton Spur,Kirkland,4347 9349 7104 3140,04/24
2,Lucius,Flynn,90,Special Needs,+1-910-584-5564,984 Toyon Heights,Salinas,4200 0547 4185 8796,04/24
3,Ryan,Ellison,106,Chartered,+16272134101,180 Newburg Landing,Tooele,3768 284910 94926,01/20
4,Bradley,Conrad,21,Marketing Manager,+1-603-053-3498,568 Onique Heights,Horn Lake,2465 0022 0910 3472,03/16
